In [1]:
import torch
from ofa.imagenet_classification.elastic_nn.networks import OFAResNets
from CNN_Pruning_Engine.Models.Resnet import ResNet101

model1 = OFAResNets(
        n_classes=100,
        bn_param=(0.1, 1e-5),
        dropout_rate=0.1,
        depth_list=1,
        expand_ratio_list=4,
        width_mult_list=1.0, 
    )
model2 = ResNet101(100)

weight = torch.load("weights/Model@ResNet101_ACC@79.89.pt")["state_dict"]
model2.load_state_dict(weight)

<All keys matched successfully>

In [2]:
model1.input_stem[0].conv.conv.load_state_dict(model2.conv1.state_dict())
model1.input_stem[0].bn.bn.load_state_dict(model2.bn1.state_dict()) 

layersx = [
            model2.layer1,
            model2.layer2,
            model2.layer3,
            model2.layer4
        ]
model2_blocks = []
for layers in layersx:
    for layer in layers:
        model2_blocks.append(layer)



for i,block in enumerate(model1.blocks,0):
    model1.blocks[i].conv1.conv.conv.load_state_dict(model2_blocks[i].conv1.state_dict())
    model1.blocks[i].conv1.bn.bn.load_state_dict(model2_blocks[i].bn1.state_dict())

    model1.blocks[i].conv2.conv.conv.load_state_dict(model2_blocks[i].conv2.state_dict())
    model1.blocks[i].conv2.bn.bn.load_state_dict(model2_blocks[i].bn2.state_dict())

    model1.blocks[i].conv3.conv.conv.load_state_dict(model2_blocks[i].conv3.state_dict())
    model1.blocks[i].conv3.bn.bn.load_state_dict(model2_blocks[i].bn3.state_dict())

    if len(model2_blocks[i].shortcut) > 0:
        model1.blocks[i].downsample.conv.conv.load_state_dict(model2_blocks[i].shortcut[0].state_dict())
        model1.blocks[i].downsample.bn.bn.load_state_dict(model2_blocks[i].shortcut[1].state_dict())
model1.classifier.linear.linear.load_state_dict(model2.linear.state_dict())

<All keys matched successfully>

In [3]:
torch.save({"state_dict":model1.state_dict()},"ResNet101OFA_ACC@79.89.pt")